<a href="https://colab.research.google.com/github/carolynw898/STAT946Proj/blob/main/DiffuSym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from models import SymbolicDiffusion, PointNetConfig, tNet
import numpy as np
from utils import processDataFiles, CharDataset
import glob
import random
from torch.utils.data import DataLoader
import tqdm

In [2]:
# setting hyperparameters
max_seq_len = 32         
n_embd = 512             
timesteps = 1000         
batch_size = 32
learning_rate = 1e-4
num_epochs = 10
blockSize = 32
numVars = 1
numYs = 1
numPoints = 250
const_range = [-2.1, 2.1]
trainRange = [-3.0, 3.0]
decimals = 8
addVars = False

In [3]:
# loading data
# path = "/kaggle/input/1-var-dataset/1_var_train.json"
path = "0_1_0_13062021_174033.json"
files = glob.glob(path)
text = processDataFiles(files)
chars = sorted(list(set(text))+['_','T','<','>',':']) # extract unique characters from the text before converting the text to a list, # T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
trainText = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(trainText) # shuffle the dataset, it's important specailly for the combined number of variables experiment
train_dataset = CharDataset(text, blockSize, chars, numVars=numVars, 
                numYs=numYs, numPoints=numPoints, addVars=addVars,
                const_range=const_range, xRange=trainRange, decimals=decimals, augment=False)

vocab_size = train_dataset.vocab_size    

loader = DataLoader(train_dataset, batch_size = batch_size)

data has 967 examples, 52 unique.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pconfig = PointNetConfig(
    embeddingSize=n_embd,
    numberofPoints=250,
    numberofVars=1,
    numberofYs=1,
)
model = SymbolicDiffusion(
    pconfig=pconfig,
    vocab_size=vocab_size,
    max_seq_len=max_seq_len,
    padding_idx=train_dataset.paddingID, # one of the dataset properties
    max_num_vars=9,
    n_layer=6,
    n_head=8,
    n_embd=n_embd,
    timesteps=timesteps,
    beta_start=0.0001,
    beta_end=0.02,
).to(device)

optimizer = Adam(model.parameters(), lr=learning_rate)

def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for _,  tokens, points, variables in loader:
        points = points.to(device)        
        tokens = tokens.to(device)        
        variables = variables.to(device)  

        optimizer.zero_grad()
        t = torch.randint(0, model.timesteps, (tokens.shape[0],), device=device)
        y_pred, noise_pred, noise = model(points, tokens, variables, t)
        loss = model.loss_fn(noise_pred, noise, y_pred, tokens, t)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(loader)
    return avg_loss

def validate_epoch(model, loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for _,  tokens, points, variables in loader:
            points = points.to(device)
            tokens = tokens.to(device)
            variables = variables.to(device)

            y_pred, noise_pred, noise = model(points, tokens, variables)
            t = torch.randint(0, model.timesteps, (tokens.shape[0],), device=device)
            loss = model.loss_fn(noise_pred, noise, y_pred, tokens, t)
            total_loss += loss.item()
    avg_loss = total_loss / len(loader)
    return avg_loss

In [5]:
# training loop
for epoch in range(num_epochs):
    # track epochs
    print(f"Epoch {epoch + 1}/{num_epochs}")
    
    # train
    train_loss = train_epoch(model, loader, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}")

torch.save(model.state_dict(), "symbolic_diffusion_model.pth")

Epoch 1/10


RuntimeError: The size of tensor a (6) must match the size of tensor b (32) at non-singleton dimension 1